In [1]:
import numpy as np
import math

import xarray as xr


import matplotlib.pyplot as plt
import cartopy.crs as crs
import cartopy.feature as cfeature
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker
import matplotlib.gridspec as gridspec

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

from IPython.display import Image

from tqdm import tqdm
import os 

g = 9.8

In [2]:
Year = "2005"

Months = []
for imon in range(1,13): ### shw
    Months.append(str(imon).zfill(2))
    
Days = []
for iday in range(1,30,3):
    Days.append(str(iday).zfill(2))
    
print(Year)
print(Months)
print(Days)

2005
['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
['01', '04', '07', '10', '13', '16', '19', '22', '25', '28']


In [3]:
directory = '/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Simulation_0.2um_Publication/6_altitudes/'+Year+'/'
# directory = '/n/home12/hongwei/Hongwei_holyscratch01/Simulation_0.2um/'+Year+'/'


N_head = 5 # first 5 lines are head lines, not include data
N_column = 4 # time, lon, lat, lev

# Nx = 36
# Ny = 21
# Nz = 5
# N_parcel = 3780

Nx = 24
Ny = 21
Nz = 6
N_parcel = Nx*Ny*Nz

if Nx*Ny*Nz!=N_parcel: print('ERROR: parcel number is wrong!')

In [4]:
# loop for all traj files

for month in Months:
    
    # read tropopause height
    TROP_folder = "/n/home12/hongwei/HONGWEI/lagranto_era5_0.2um/Plot_python/Tropopause_height/"
    TROP_ds  = xr.open_dataset(TROP_folder+'ERA5_monmean_'+month+'.nc')
    TROP_p   = TROP_ds["wmo_1st_p"][:,:,:] # [time: 1, lat: 601, lon: 1200]
    TROP_lon = TROP_ds["lon"].values
    TROP_lat = TROP_ds["lat"].values
    
    
    for day in tqdm(Days):
    
        # (1) read original data from traj files
        filename = "traj_"+Year+month+day+".1"
        file1 = open(directory+filename, 'r')
        Lines = file1.readlines()

        Nt = int( (len(Lines)-4)/N_parcel - 1 )
                
        data = np.zeros((N_parcel, Nt, N_column))

        count = 0
        # Strips the newline character
        for line in Lines:
            count += 1
    
            if count>=5:
                i = count-5
                i_parcel = math.floor( i / (Nt+1) ) # Nt time lines plue 1 empty line
                i_t = i%(Nt+1)
        
                if i_t!=0:
                    a = line.split()

                    if i_t==1:
                        if float(a[0])!=0.0: print('ERROR: first time is not 0 !!!')
                
                    data[i_parcel,i_t-1,0] = float(a[0]) # t [h]
                    data[i_parcel,i_t-1,1] = float(a[1]) # lon
                    data[i_parcel,i_t-1,2] = float(a[2]) # lat
                    data[i_parcel,i_t-1,3] = float(a[3]) # lev

                                    
        # (2) re-shape data(N_parcel, Nt, N_column) to data2(Nz,Ny,Nx,Nt,N_column)
        data2 = np.zeros((Nz,Ny,Nx,Nt,N_column))

        for ix in range(Nx):
            for iy in range(Ny):
                for iz in range(Nz):
                    i_parcel = ix*(Ny*Nz) + iy*Nz + iz
                    data2[iz,iy,ix,:,:] = data[i_parcel,:,:]
                
                        
        # (3) calculate injected tracer lifetime in the stratosphere
        lifetime   = np.zeros((Nz,Ny,Nx))
        lifetime_N = np.zeros((Nz,Ny,Nx))
        lifetime_S = np.zeros((Nz,Ny,Nx))

        # tropical area exiting location 
        Lon_sink = np.zeros((Nz,Ny,Nx))
        Lat_sink = np.zeros((Nz,Ny,Nx))
        Lev_sink = np.zeros((Nz,Ny,Nx))
                
        # select the min pressure that the injected particle reaches overal
        P_min = np.zeros((Nz,Ny,Nx))
        
        for iy in range(Ny):
            for iz in range(Nz):
                for ix in range(Nx):
                    Num = 0
                    Num_N = 0
                    Num_S = 0
                    for it in range(Nt-4):
                        Num = Num+1
                    
                        if data2[iz,iy,ix,it,2]>=0: # lat>=0
                            Num_N = Num_N + 1
                        else:
                            Num_S = Num_S + 1
                            
                            
                        # check whether touch the tropopause
                        LON_1 = data2[iz,iy,ix,it,1]
                        
                        LAT_1 = data2[iz,iy,ix,it,2]
                        LAT_2 = data2[iz,iy,ix,it+1,2]
                        LAT_3 = data2[iz,iy,ix,it+2,2]
                        
                        P_1   = data2[iz,iy,ix,it,3]
                        P_2   = data2[iz,iy,ix,it+1,3]
                        P_3   = data2[iz,iy,ix,it+2,3]
                    
                    
                        # (1) once lower than 90 hPa, begin to check whether touch tropopause
                        if P_1>85.0: 
                        
                            # match location
                            
                            # make sure LON_1 is alway in the same range of TROP_lon (i.e., -180 to 180)
                            if LON_1<np.min(TROP_lon): LON_1 = LON_1+360
                            if LON_1>np.max(TROP_lon): LON_1 = LON_1-360
                            ilon   = (np.abs(TROP_lon - LON_1)).argmin() 
                    
                            ilat   = (np.abs(TROP_lat - LAT_1)).argmin()

                            P_trop = TROP_p[0, ilat, ilon]
                        
                        
                            # compare particle pressure with tropopause pressure
                            if P_1>P_trop and P_2>P_trop and P_3>P_trop: 
                                lifetime[iz,iy,ix] = Num # [day]
                                lifetime_N[iz,iy,ix] = Num_N
                                lifetime_S[iz,iy,ix] = Num_S
                            
                                Lon_sink[iz,iy,ix] = data2[iz,iy,ix,it,1]
                                Lat_sink[iz,iy,ix] = data2[iz,iy,ix,it,2]
                                Lev_sink[iz,iy,ix] = data2[iz,iy,ix,it,3]
                            
                                P_min[iz,iy,ix] = data2[iz,iy,ix,0,3]
                                if it>1: P_min[iz,iy,ix] = np.min(data2[iz,iy,ix,0:it,3])
                                
                                break
                                
                        # (2) once exiting tropical area, break
                        if abs(LAT_1)>=30.0 and abs(LAT_2)>=30.0 and abs(LAT_3)>=30.0:
                                
                            lifetime[iz,iy,ix] = Num # [day]
                            lifetime_N[iz,iy,ix] = Num_N
                            lifetime_S[iz,iy,ix] = Num_S
                            
                            Lon_sink[iz,iy,ix] = data2[iz,iy,ix,it,1]
                            Lat_sink[iz,iy,ix] = data2[iz,iy,ix,it,2]
                            Lev_sink[iz,iy,ix] = data2[iz,iy,ix,it,3]
                            
                            P_min[iz,iy,ix] = data2[iz,iy,ix,0,3]
                            if it>1: P_min[iz,iy,ix] = np.min(data2[iz,iy,ix,0:it,3])
                                
                            break
                                
        
        file1.close()
        
        
        
        # save lifetime_01[Nz,Ny,Nx,N_inject] to Lifetime01.txt
        
        with open('./Exiting_Pressure/Exiting_Press_'+Year+month+day+'.txt', 'w') as f:
            for ix in range(Nx):
                for iy in range(Ny):
                    for iz in range(Nz):
                                                
                        f.write(  str(lifetime[iz,iy,ix])   + ',' \
                                + str(lifetime_N[iz,iy,ix]) + ',' \
                                + str(lifetime_S[iz,iy,ix]) + ',' \
                                
                                + str(data2[iz,iy,ix,0,1])  + ',' \
                                + str(data2[iz,iy,ix,0,2])  + ',' \
                                + str(data2[iz,iy,ix,0,3])  + ',' \
                                
                                + str(Lon_sink[iz,iy,ix])  + ',' \
                                + str(Lat_sink[iz,iy,ix])  + ',' \
                                + str(Lev_sink[iz,iy,ix])  + ',' \
                                
                                + str(P_min[iz,iy,ix])  )
                        
                        f.write('\n')
        f.close()


100%|██████████| 10/10 [06:15<00:00, 37.58s/it]
